<a href="https://colab.research.google.com/github/IvanNece/LABS_Edge-Computing-Systems-for-AI-and-ML/blob/main/Lab_0/1_BostonHousingMarket/RegressionAndRegularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear and Polynomial Regression and regularization
As we saw in our lessons, linear regression is a good model only when there is actually a linear dependence from the features to the target.

In some cases, for example when the target is a sinusoidal function of the features, a first degree polynomial is not a sufficiently good approximation. In this tutorial we will learn how to create a good polynomial model and how to tune it with regularization.

## Importing packages and functions
- [numpy](https://numpy.org) is a python package that provides support for more efficient numerical computation
- [pandas](https://pandas.pydata.org) is a convenient library that supports dataframes. Pandas is technically optional because [Scikit-Learn](https://scikit-learn.org) can handle numerical matrices directly, but it will make our lives easier, as we will see later.
- [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) is a class that implements the Linear Regression model (all classes in scikit-learn have capital names)
- [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) implements the Ridge regularization method for Linear Regression, that adds to the loss function the sum of the squares of the weigths (i.e. minimizes the L2 norm of the vector of weights).
- [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) implements the Lasso regularization method for Linear Regression, that adds to the loss function the sum of the modules of the weigths (i.e. minimizes the L1 norm of the vector of weights).
- [mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) is the usual Mean Square Error loss function used for Linear Regression (all functions in scikit-learn have lower case names).
- [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) splits the dataset into a training set and a validation set (the functionality is the same for test and validation sets, hence the name; here we will use it for the validation set only).
- [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) creates various powers of the input features and theor cross-products, as a mapping function for Polynomial Regression
- [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) is a utility that can be used to concatenate various ML steps into a single pipeline.
- [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) scales your data based on mean and standard deviation, so that it fits in a small interval (e.g. [-1...1]) centered around 0.

In [1]:
import numpy as np
import pandas as pd
import math

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

##  Importing data
Let us now read the data for this lab. They cover the Boston housing market (the original is [here](https://www.kaggle.com/c/boston-housing)).

As the data is stored in a .csv file, we use the `read_csv()` function from pandas to read it in, show the data format, and print the first three lines.

A "[dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)" in pandas (and in other ML frameworks such as R) is a class that includes
- a data set (in this case 14 columns and 506 lines)
- some format information, such as the column names (from the xlsx header), the data types and so on
- methods to print mean, standard deviation and so on.

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/lavagno/eesam/main/housing.csv')
print("Some generic information about the dataframe:")
print(train_df.info())
print("The first three rows of the dataframe:")
print(train_df.head(3))
print("The mean of each column:")
print(train_df.mean())
print("The standard deviation of each column:")
print(train_df.std())

Some generic information about the dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB
None
The first three rows of the dataframe:
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  2

The meanings of the data columns are:
- CRIM per capita crime rate by town
- ZN proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS proportion of non-retail business acres per town
- CHAS Charles River = 1 if tract bounds river; 0 otherwise
- NOX nitric oxides concentration (parts per 10 million)
- RM average number of rooms per dwelling
- AGE proportion of owner-occupied units built prior to 1940
- DIS weighted distances to five Boston employment centres
- RAD index of accessibility to radial highways
- TAX full-value property-tax rate per 10,000 dollars
- PTRATIO pupil-teacher ratio by town
- B is a function of the proportion of blacks by town
- LSTAT % lower status of the population
- MEDV Median value of owner-occupied homes in 1000's of dollars, and this is the **target** of the model.



## Training Linear Regression
Let us split our data into a training set and a validation set. We will hold out 30% of the data for validation. We will use a **fixed initial random state** to make our experiment reproducible (in practice, you should never do that, except for model debugging).

The `drop()` method of the dataframe simply removes the named column, and it is much easier to use than listing explicitly all the features.

In [3]:
X = train_df.drop('MEDV', axis=1)
y = train_df['MEDV']

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.3)

Let us establish a baseline by training a linear regression model.

The `fit()` function of the `LinearRegression` class performs the training, and the `predict()` function evaluates the model. This is then used to compute our usual loss function (mean square error) and its root (which is directly comparable with the mean house price) for both testing and validation.

Finally we also print the [$R^2$ score of the prediction](https://en.wikipedia.org/wiki/Coefficient_of_determination), which is a standard quality index used in statistics. It is similar to the training accuracy measure and it roughly tells how much of the data variance is predicted by the model. It is a number from 0 to 1, and the closer it is to 1, the better our model is.
- **Low values of $R^2$ for both sets** are an indication of **underfitting**.
- **Lower values of $R^2$ for the validation set than for the training set** are an indication of **overfitting**

In [4]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_train = lr_model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = math.sqrt(mse_train)
print('Training MSE: ', mse_train, ' RMSE: ', rmse_train)

y_pred_val = lr_model.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = math.sqrt(mse_val)
print('Validation MSE: ', mse_val, ' RMSE: ', rmse_val)

print('Training score: {}'.format(lr_model.score(X_train, y_train)))
print('Validation score: {}'.format(lr_model.score(X_val, y_val)))

Training MSE:  22.545481487421423  RMSE:  4.748208239685937
Validation MSE:  21.517444231177183  RMSE:  4.638689926172818
Training score: 0.7434997532004697
Validation score: 0.7112260057484936


As you can see, MSEs, RMSEs and scores are not too far from each other. This means that the model is **not grossly overfitting**.

On the other hand
- both RMSEs, around 4.5, are relatively large with respect to the mean house value (MEDV) of 22, and they are not too far from the standard deviation, which is 9.
- both scores are around 0.7, which is low.
This means that the model is **probably underfitting**.

The only way to "prove" and **solve** underfitting, is to try a more complex model, with more parameters, and hence better able to model the underlying data population.



## Polynomial Regression
We will use **Polynomial Regression**, i.e. linear regression with a set of polynomial mapping functions (in general **Generalized Linear Regression** can use functions of the features such as sine, cosine, exponential, etc.).
Thankfully, scikit-learn has an implementation for this and we do not need to do it manually. It is the [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) class.

Note that in our lessons we saw polynomial features only of one feature. When there are multiple features, **also the cross-products**, up to the specified degree, will be inserted. For example, if there are two features `[a,b]` the resulting polynomial feature vector will be:
```
[1, a, b, a^2, ab, b^2]
```
Since we have 13 features in our dataset, we have 13 features for the 1st degree, 13 for the 2nd degree, and 13*2/2 for the mixed products, plus the constant term, i.e. 105 features and weights in total.

## Scaling
Something else that we would like to do is **scale** our data to a range between 0 and 1. This serves the purpose of letting us work with **reasonable numbers when we raise a feature to a power**. As we saw, it also improves the convergence speed for the model, albeit this not really a problem in this case, since the dataset is really small.



## Pipelining
Finally, because we need to carry out the same operations on our training and validation (and in real practice test) sets, we will introduce a **pipeline** (conceptually similar to a unix pipeline between processes, or a processor pipeline). This will let us pipe the work done by the methods of our classes, so that the same steps get carried out repeatedly for every variation of the hyperparameters that we want to tune.
This is one of the cases in which using an object-oriented language greatly helps, since if all the stages of a pipeline derive from a few parent classes, the pipeline can be designed to work by calling the "right" method of each stage.

To summarize, we will scale our data, then create polynomial features up to the second degree, and then train a linear regression model.

Very conveniently, the pipeline class itself **inherits** the fit and predict methods from the LinearRegression class that it uses, hence we will be able to call them directly from the pipeline, rather than having to access the underlying model instance.

In [5]:
steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', LinearRegression())
]

pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)

y_pred_train = pipeline.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = math.sqrt(mse_train)
print('Training MSE: ', mse_train, ' RMSE: ', rmse_train)

y_pred_val = pipeline.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = math.sqrt(mse_val)
print('Validation MSE: ', mse_val, ' RMSE: ', rmse_val)

print('Training score: {}'.format(pipeline.score(X_train, y_train)))
print('Validation score: {}'.format(pipeline.score(X_val, y_val)))

Training MSE:  4.660318639997241  RMSE:  2.158777116794886
Validation MSE:  25.257540307734835  RMSE:  5.025688043217052
Training score: 0.9469794920108198
Validation score: 0.6610321968877256


## Analyzing the polynomial regression results and regularization
As expected, the training MSE and RMSE decrease significantly, and the training score increases significantly because the model, being more complex, can better fit the **training** data (but, as we saw, may overfit).

But the validation errors and the validation score become much worse. This means that we are now **overfitting**, and now is the time to use **regularization** to reduce its impact.

Remember that **some initial overfitting is good**, because it allows us to use regularization to **automatically search for the best model**.

We will use Ridge regularization for Linear Regression (also called simply Ridge Regression), which adds the squares of the weights to the loss function.

Note that the regularization strength, i.e. the coefficient used to multiply the sum of the squares of the weights, that we called $\lambda$ in our lessons, is called `alpha` in scikit-learn.


In [8]:
ridge_steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', Ridge(alpha=0.1))
]

ridge_pipe = Pipeline(ridge_steps)
ridge_pipe.fit(X_train, y_train)

y_pred_train = ridge_pipe.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = math.sqrt(mse_train)
print('Training MSE: ', mse_train, ' RMSE: ', rmse_train)

y_pred_val = ridge_pipe.predict(X_val)
mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = math.sqrt(mse_val)
print('Validation MSE: ', mse_val, ' RMSE: ', rmse_val)

print('Training Score: {}'.format(ridge_pipe.score(X_train, y_train)))
print('Validation Score: {}'.format(ridge_pipe.score(X_val, y_val)))

Training MSE:  4.711271197926952  RMSE:  2.170546290205982
Validation MSE:  24.062638849089595  RMSE:  4.905368370376438
Training Score: 0.9463998040724038
Validation Score: 0.6770683238200299


Now all the measures are **better than with only linear regression**. This means that the target is sensitive to the square or the cross-product of **some features**, but possibly not all of them.


## Analyzing the results of regularization
Let us now print the weights with these two commands.
- The first one "extracts" the model step from the pipeline.
- The second one prints the 105 weights.

Note how almost none of the weights are close to zero. If we want to bring them closer to zero, we must:
- either increase `alpha` as long as the validation error does not increase "too much",
- or use another regularization method (the Lasso Regularization mentioned below) whose goal is actually to **force** some weights to become zero.

On the other hand, the goal of Ridge is simply to **improve the generalization power of the model**, based on the empirical observation that large weights "hurt" generalization, without really forcing them to zero.

In [9]:
lr=ridge_pipe.named_steps['model']
print(lr.coef_.size)
print(lr.coef_)

105
[ 0.          3.53826063 -1.5562031   2.41129376  0.03784823 -1.0290763
  3.67969521 -1.88989518 -0.8499394  -2.21599825  1.57888679 -0.35053083
  1.05644525 -3.56831357  0.13259235 -1.17194057  4.20862862  5.3668381
 -1.95694487  1.12483705  0.54014092  2.30963112 -4.24347606 -2.30728568
  4.25798253 -0.41059196  2.16307096 -0.18992476  0.03155989 -0.23787267
 -1.96456679  0.26647517  0.35611705 -0.81680707 -3.07992388  3.21052917
  0.16049884 -0.69912577 -1.30045339  2.06208986 -0.53547895  1.00575246
  1.92362506  0.5047473   1.84036601  0.64618498  0.10475483 -0.20796908
  1.388942   -0.70518646  0.12377384 -1.07839172 -1.12242286 -0.67758622
 -1.54671357  0.55451718 -0.0229335  -0.83571554  2.61894748 -0.86166319
 -1.68780096 -0.57702985  0.34666214  2.32717122 -0.64074245  3.05318031
 -3.00227495 -0.59939699  0.97799386  0.67156665 -1.68733215 -0.77027621
 -0.70727218 -2.70224539 -0.56054023 -0.4407643   0.15312699 -0.2117823
  0.43087444  3.04940386 -2.10943667 -0.14578831 -

## Your tasks
1. Find (by trial and error) the "best" value of `alpha` that leads to the smallest validation error.
1. Try another form of regularization, namely the Lasso, which adds the modules of the weights (i.e. minimizes the L1 norm of the vector of weights), rather than their squares. See if:
 - it improves the validation performance,
 - it makes some weights closer to zero (with Ridge Regression, the smallest one was $-4\cdot10^{-4}$).

Note that
- the `alpha` coefficient for `Ridge` can be arbitrary but must be non-negative (values that are too large or too small may be sub-optimal, of course).
- the `alpha` coefficient for Lasso must be between 0 and 1, otherwise the method may not converge to a reasonable model.
- with `alpha` (i.e. $\lambda$) =0 both Ridge and Lasso are simply Linear Regression, of course.
- there is a third regularization mechanism, namely [`ElasticNet`](scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) that combines the "best" features of both `Lasso` and `Ridge`, using two separate coefficients to minimize both the L1 and L2 norms of the weights. In this particular case, it does not improve things dramatically, but you can try it out if you are curious (as you should be :-) ).

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np, math

alphas = [0, 1e-4, 1e-3, 1e-2, 0.1, 0.3, 1]

best = {
    'alpha': None, 'rmse_val': float('inf'),
    'rmse_train': None, 'r2_train': None, 'r2_val': None,
    'nonzero': None, 'min_abs': None
}

print("=== LASSO sweep su alpha ===")
for alpha in alphas:
    lasso_steps = [
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scalar', StandardScaler()),
        ('model', Lasso(alpha=alpha, max_iter=50000))
    ]
    lasso_pipe = Pipeline(lasso_steps)
    lasso_pipe.fit(X_train, y_train)

    # metriche train
    y_pred_train = lasso_pipe.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = math.sqrt(mse_train)
    r2_tr = r2_score(y_train, y_pred_train)

    # metriche validation
    y_pred_val = lasso_pipe.predict(X_val)
    mse_val = mean_squared_error(y_val, y_pred_val)
    rmse_val = math.sqrt(mse_val)
    r2_v = r2_score(y_val, y_pred_val)

    # sparsità dei pesi
    coefs = lasso_pipe.named_steps['model'].coef_
    nonzero = int(np.sum(np.abs(coefs) > 1e-8))
    min_abs = float(np.min(np.abs(coefs))) if coefs.size > 0 else float('nan')

    print(f"\nalpha = {alpha}")
    print('Training MSE: ', mse_train, ' RMSE: ', rmse_train)
    print('Validation MSE: ', mse_val, ' RMSE: ', rmse_val)
    print('Training Score:', r2_tr)
    print('Validation Score:', r2_v)
    print(len(coefs))
    print('Non-zero weights:', nonzero)
    print('Smallest abs weight:', min_abs)

    # aggiorna best
    if rmse_val < best['rmse_val']:
        best.update({
            'alpha': alpha, 'rmse_val': rmse_val,
            'rmse_train': rmse_train, 'r2_train': r2_tr, 'r2_val': r2_v,
            'nonzero': nonzero, 'min_abs': min_abs
        })

print("\n=== MIGLIORE (in base a RMSE validation) ===")
print(f"alpha = {best['alpha']}")
print('Training RMSE:', best['rmse_train'], ' | Validation RMSE:', best['rmse_val'])
print('Training R2  :', best['r2_train'],   ' | Validation R2  :', best['r2_val'])
print('Non-zero weights:', best['nonzero'],  '| Smallest abs weight:', best['min_abs'])


=== LASSO sweep su alpha ===


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.352e+02, tolerance: 3.112e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent


alpha = 0
Training MSE:  4.718867721171118  RMSE:  2.1722954958225915
Validation MSE:  23.81727162181733  RMSE:  4.880294214677772
Training Score: 0.9463133783250524
Validation Score: 0.6803612648178816
104
Non-zero weights: 104
Smallest abs weight: 0.04613239722169756


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.497e+02, tolerance: 3.112e+00
  model = cd_fast.enet_coordinate_descent(



alpha = 0.0001
Training MSE:  4.737786771177615  RMSE:  2.176645761527956
Validation MSE:  23.105678504440018  RMSE:  4.806836642162912
Training Score: 0.9460981360380963
Validation Score: 0.6899111716088187
104
Non-zero weights: 103
Smallest abs weight: 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.329e+01, tolerance: 3.112e+00
  model = cd_fast.enet_coordinate_descent(



alpha = 0.001
Training MSE:  5.008449599865825  RMSE:  2.237956567913199
Validation MSE:  20.735343423290836  RMSE:  4.553607737090541
Training Score: 0.943018801387527
Validation Score: 0.7217221581620517
104
Non-zero weights: 85
Smallest abs weight: 0.0

alpha = 0.01
Training MSE:  6.881833955728284  RMSE:  2.6233249809599046
Validation MSE:  16.759102528464187  RMSE:  4.093788285740261
Training Score: 0.9217052823173225
Validation Score: 0.7750851390518344
104
Non-zero weights: 50
Smallest abs weight: 0.0

alpha = 0.1
Training MSE:  13.960382781035289  RMSE:  3.736359562600378
Validation MSE:  15.426913616896794  RMSE:  3.9277109894818882
Training Score: 0.8411725369116961
Validation Score: 0.7929637267204144
104
Non-zero weights: 24
Smallest abs weight: 0.0

alpha = 0.3
Training MSE:  18.16082464543094  RMSE:  4.261551905753459
Validation MSE:  18.754312950490647  RMSE:  4.330625006911895
Training Score: 0.7933840531977566
Validation Score: 0.748308497888011
104
Non-zero weights: 

## Analysis hints
You may (should? :-) ) have noticed that Lasso regularization actually forces several weights to be **zero**, not just small.
It is one of the mechanisms (in addition to Principal Component Analysis and Singular Value Decomposition that we will see among the unsupervised learning approaches) to identify **which features are least important, or not important at all** for a given target of a given dataset.

Credits: the lab was mostly taken from [here](https://medium.com/coinmonks/regularization-of-linear-models-with-sklearn-f88633a93a2).